In [13]:
import openai
import json
import os

# Use your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")  # Or set your API key directly


In [18]:
import os
from openai import OpenAI

# os.environ["OPENAI_API_KEY"] = "sk-proj-If7SKDdQbA1nHIeZfRyYT3BlbkFJTaFyceLNDnp6zTaJ0yiv"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [19]:
class Conversation:
    def __init__(self, log_file="convo.log"):
        self.log_file = log_file
        self.conversation_log = []
        self.load_conversation_from_file()

    def load_conversation_from_file(self):
        """Load conversation history from a log file if it exists."""
        try:
            with open(self.log_file, 'r') as file:
                self.conversation_log = json.load(file)
        except FileNotFoundError:
            # No previous conversation found, start with an empty list
            self.conversation_log = []

    def save_conversation_to_file(self):
        """Save the current conversation history to a log file."""
        with open(self.log_file, 'w') as file:
            json.dump(self.conversation_log, file, indent=4)

    def get_conversation_history(self):
        """Build the conversation history to send to the LLM."""
        # Combine all previous queries and responses into one prompt
        history = ""
        for exchange in self.conversation_log:
            history += f"User: {exchange['query']}\n"
            history += f"Assistant: {exchange['response']}\n"
        return history

    def call_openai_api(self, query):
        """Make a call to the OpenAI API with the current query and previous history."""
        # Build the prompt from the conversation history + new user query
        conversation_history = self.get_conversation_history()
        prompt = f"{conversation_history}\nUser: {query}\nAssistant:"

        # Call the OpenAI API
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            prompt=prompt,
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.7
        )
        llm_response = response.choices[0].text.strip()

        # Add the new query and response to the conversation log
        self.conversation_log.append({
            "query": query,
            "response": llm_response
        })

        # Save the updated conversation log to file
        self.save_conversation_to_file()

        return llm_response

In [22]:
import openai

# gets API Key from environment variable OPENAI_API_KEY
client = openai.OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

print("Run completed with status: " + run.status)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)

    print("messages: ")
    # for message in messages:
    #     assert message.content[0].type == "text"
    #     print({"role": message.role, "message": message.content[0].text.value})
    print(messages)


Run completed with status: completed
messages: 
SyncCursorPage[Message](data=[Message(id='msg_4jMyLG6QDW5FMA1pzpnTrOGO', assistant_id='asst_Dr0CdIdsn2B8EkL9ZFIRyZhx', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).'), type='text')], created_at=1729257158, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_dVhEndY6mU9XnpEsVnjaX3LG', status=None, thread_id='thread_nETCT1JKScT8y2ez16IoE2Bj'), Message(id='msg_RzFWl3zEq7kYq7jQe2WAGqsq', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1729257144, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_nETCT1JKScT8y2ez16IoE2Bj')], object=

In [ ]:
import openai
import os
import time

client = openai.OpenAI(api_key="sk-proj-If7SKDdQbA1nHIeZfRyYT3BlbkFJTaFyceLNDnp6zTaJ0yiv")

# Create a math tutor assistant (you can modify this depending on the assistant's function)
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer math questions and run code if necessary.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o",
)

# Start a new conversation thread
thread = client.beta.threads.create()

def send_message(thread_id, assistant_id, user_message):
    # User sends a message to the assistant
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_message,
    )

    # Poll the assistant for a response
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions="Please address the user by their name and solve their math problems.",
    )

    return run

import json

def save_conversation_to_json(messages):
    for msg in messages:
        assert msg.content[0].type == "text"
        conversation_data = [
            f"{msg.role}: {msg.content[0].text.value}"
        ]
        with open("conversation.json", "w") as f:
            json.dump(conversation_data, f, indent=4)

def real_time_conversation():
    print("You are now conversing with the Math Tutor bot. Type 'exit' to quit.")

    while True:
        # Get user input
        user_message = input("You: ")

        if user_message.lower() == "exit":
            print("Ending conversation.")
            break
        
        # Send message and get the response from the assistant
        run = send_message(thread.id, assistant.id, user_message)

        # Wait until the response is ready
        while run.status != "completed":
            print("Waiting for assistant's response...")
            time.sleep(1)  # Pause for a bit before checking again

        # Once completed, retrieve the messages from the thread
        messages = client.beta.threads.messages.list(thread_id=thread.id)
                                                     
        # Print out the assistant's latest response
        latest_message = list(messages)[-1]
        assert latest_message.content[0].type == "text"
        print(f"Assistant: {latest_message.content[0].text.value}")

        save_conversation_to_json(messages)

real_time_conversation()
client.beta.assistants.delete(assistant.id)

You are now conversing with the Math Tutor bot. Type 'exit' to quit.


In [10]:
# Example usage:
if __name__ == "__main__":
    # Initialize the Conversation class
    convo = Conversation()

    # User inputs a question
    user_input = "What are some ways to manage stress?"

    # Call the OpenAI LLM and get a response, using the full conversation history
    llm_response = convo.call_openai_api(user_input)

    # Output the LLM's response
    print("LLM Response:", llm_response)

    # For debugging, you can also check the full conversation log
    print("Full Conversation Log:", convo.conversation_log)

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
